In [1]:
import numpy as np
import pandas as pd
import re, collections, nltk, math
import sklearn.feature_extraction.text as featExt
from nltk.tokenize import RegexpTokenizer
from sklearn.preprocessing import normalize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.corpus import wordnet as wn
from nltk.tokenize import PunktSentenceTokenizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from os import path
from nltk.util import ngrams
from sklearn.decomposition import PCA
import lda
import random
from scipy import cluster
from matplotlib import pyplot

Goal: refactor code. It's too damn messy and is slowing me down

In [2]:
def isASCIIString(string):
    try:
        string.decode('ascii')
        return True
    except:
        return False

# Just some ASCII coding issues
def replaceString(string):
    return string.replace("\\n", " ").replace("&amp;", "&").replace('&#039;', '\'').replace("&quot;", "\"").replace("&lt;", "<").replace("&gt;", ">").replace("\xe2\x80\x93", "-").replace("\xe2\x80\x99", "\'").strip()


def is_eligible_word(word, stopWordsRemoved = True, puncRemoved = True, regexPattern = "^[^a-zA-Z0-9]+", lang = "english", specialNotToRemove = set([])):
    specialToRemove = set(["even"]) # Feel free to edit ["n't"]
    if puncRemoved and re.match(regexPattern, word): 
        return False
    elif word in specialToRemove: 
        return False
    elif stopWordsRemoved and word in set(stopwords.words(lang)) and word not in specialNotToRemove:
        return False
    return True


def handle_negations(wordArray, handleNegations = True):
    '''
        Everytime you see a not (n't - but change n't to not), fuse it with the word after. 
    '''
    if not handleNegations or len(wordArray) == 0 or (type(wordArray[0]) in set([list, dict, set, tuple])):
        return wordArray
    else:
        for i in range(len(wordArray)):
            if wordArray[i].lower() in set(["n\'t", "n\"t", "no", "didnt"]) :
                wordArray[i] = "not"
            if wordArray[i] == "not" and i < (len(wordArray) - 1):
                wordArray[i + 1] = "not_" + wordArray[i + 1]
    return [word for word in wordArray if word != 'not']


def get_ngram_set(tokenizedMessage, ngram, isLemmatized = True, stopWordsRemoved = True, puncRemoved = True, regexPattern = "^[^a-zA-Z0-9]+", lang = "english", specialNotToRemove = set([]), handleNegations = True):
    if handleNegations:
        specialNotToRemove.add("not")
        specialNotToRemove.add("no")
    fixedTokenizedMessage = handle_negations([word for word in tokenizedMessage], handleNegations = handleNegations)
    ngramSet = [[word for word in ngramindiv if is_eligible_word(word, stopWordsRemoved = stopWordsRemoved, puncRemoved = puncRemoved, regexPattern = regexPattern, lang = lang, specialNotToRemove = specialNotToRemove)] for ngramindiv in ngrams(fixedTokenizedMessage, ngram)]
    if isLemmatized:
        wordnet_lemmatizer = WordNetLemmatizer()
        if ngram == 1:
            return [wordnet_lemmatizer.lemmatize(word) for ngramindiv in ngramSet for word in ngramindiv]
        else: 
            return [[wordnet_lemmatizer.lemmatize(word) for word in ngramindiv] for ngramindiv in ngramSet if len(ngramindiv) > 0]
    elif ngram == 1:
        return [word for ngramindiv in ngramSet for word in ngramindiv]
    else:
        return [ngramindiv for ngramindiv in ngramSet if len(ngramindiv) > 0]

    
def get_message_set(df, category, split_by = 'page_vertical_tx', messageColumnName = "message"):
    if category != None:
        df = df.loc[df[split_by] == category]
    if len(df.shape) > 0 and df.shape[0] == 0:
        print("Category \'" + str(category) + "\' has no rows. Check spelling.")
        return []
    else:
        return [replaceString(str(message)) for messageL in df[[messageColumnName]].values.tolist() for message in messageL if isASCIIString(replaceString(str(message)))]            

    
def tokenize_message_set(messageSet, isNormalized): 
    if isNormalized:
        return [nltk.word_tokenize(message.lower()) for message in messageSet]
    else: 
        return [nltk.word_tokenize(message) for message in messageSet] 
    
    
def obtain_Message_Token_List(df, category, ngram = 1, split_by = 'page_vertical_tx', messageColumnName = "message", lang = "english", regexPattern = "^[^a-zA-Z0-9]+", pos_tagset = 'universal', specialNotToRemove = set([]), isNormalized = True, isLemmatized = True, stopWordsRemoved = True, handleNegations = True, puncRemoved = True, posTag = True):
    ## Step 1: get the messages perfectly, dealing with weird cases
    messageSet = get_message_set(df, category, split_by = 'page_vertical_tx', messageColumnName = "message")
    if len(messageSet) == 0:
        return []
    else:    
        ## Step 2: Tokenize and (optionally) normalize
        messageToken = tokenize_message_set(messageSet, isNormalized)
        ## Step 3: Remove stop words, punctuation
        ## Step 4: Lemmatize and N-gram (kind of with step 3 too)
        ## Step 5: POS-tag?
        if ngram == 1 and posTag:
            return [nltk.pos_tag(get_ngram_set(message, ngram, isLemmatized = isLemmatized, stopWordsRemoved = stopWordsRemoved, puncRemoved = puncRemoved, regexPattern = regexPattern, lang = lang, specialNotToRemove = specialNotToRemove, handleNegations = handleNegations), tagset = pos_tagset) for message in messageToken]
        else:
            return [get_ngram_set(message, ngram, isLemmatized = isLemmatized, stopWordsRemoved = stopWordsRemoved, puncRemoved = puncRemoved, regexPattern = regexPattern, lang = lang, specialNotToRemove = specialNotToRemove, handleNegations = handleNegations) for message in messageToken]


This fix below basically removes all the badly formed lines. I'm too lazy to deal with so many of them individually.

In [3]:
df = pd.read_csv("comments_raw_merged_with_page_data_190716.csv", delimiter = ',')
faultyArrays = []
goodArrays = []
for i in range(len(df[["message"]].values.tolist())):
    message = df["message"].values.tolist()[i]
    if not isASCIIString(replaceString(str(message))):
        faultyArrays.append(i)
    else:
        goodArrays.append(i)
        
df = df.iloc[goodArrays,:]
## messageSet = [replaceString(str(message)) for messageL in file[["message"]].values.tolist() for message in messageL if isASCIIString(replaceString(str(message)))]

# WORD CLOUD

Attempt 1: 
1. Tokenize / Normalize / Lemmatize
2. Remove stop words and punctuation
3. POS tag
4. Count

Results are currently decent. Of course there are quite a lot of junk still floating, and the challenge is to take it away.

2027hrs: Reported seeing 'nan's and quite a number of 'n\'t's as well. As such can work to POS it.

Brute-forced n't problem.


In [5]:
dfGood = df[df["helpful_yn"] == "yes"]
dfBad = df[df["helpful_yn"] == "no"]

listOfAllWordsUnigramGood = {}
acceptableCategories = set(['NOUN'])
for category in dfGood["page_vertical_tx"].unique().tolist():
    if category != None:
        wordSet = obtain_Message_Token_List(dfGood, category, ngram = 1)
        output = sorted(Counter([word[0] for message in wordSet for word in message if word[1] in acceptableCategories]).iteritems())
        listOfAllWordsUnigramGood[category] = output

listOfAllWordsBigramGood = {}
for category in dfGood["page_vertical_tx"].unique().tolist():
    if category != None:
        wordSet = obtain_Message_Token_List(dfGood, category, ngram = 2)
        output = sorted(Counter([" ".join(word) for message in wordSet for word in message]).iteritems())
        listOfAllWordsBigramGood[category] = output

listOfAllWordsUnigramBad = {}
acceptableCategories = set(['NOUN'])
for category in dfBad["page_vertical_tx"].unique().tolist():
    if category != None:
        wordSet = obtain_Message_Token_List(dfBad, category, ngram = 1)
        output = sorted(Counter([word[0] for message in wordSet for word in message if word[1] in acceptableCategories]).iteritems())
        listOfAllWordsUnigramBad[category] = output

listOfAllWordsBigramBad = {}
for category in dfBad["page_vertical_tx"].unique().tolist():
    if category != None:
        wordSet = obtain_Message_Token_List(dfBad, category, ngram = 2)
        output = sorted(Counter([" ".join(word) for message in wordSet for word in message]).iteritems())
        listOfAllWordsBigramBad[category] = output

KeyboardInterrupt: 

In [ ]:
#d = path.dirname(__file__)

for category in listOfAllWordsUnigramGood.keys():
    if len(listOfAllWordsUnigramGood[category]) > 0:
        wc = WordCloud()
        wc.generate_from_frequencies(listOfAllWordsUnigramGood[category])
        wc.to_file((category + "_good_Unigram.png"))

for category in listOfAllWordsBigramGood.keys():
    if len(listOfAllWordsBigramGood[category]) > 0:
        wc = WordCloud(width = 800, height = 400)
        wc.generate_from_frequencies(listOfAllWordsBigramGood[category])
        wc.to_file((category + "_good_Bigram.png"))

for category in listOfAllWordsUnigramBad.keys():
    if len(listOfAllWordsUnigramBad[category]) > 0:
        wc = WordCloud()
        wc.generate_from_frequencies(listOfAllWordsUnigramBad[category])
        wc.to_file((category + "_bad_Unigram.png"))

for category in listOfAllWordsBigramBad.keys():
    if len(listOfAllWordsBigramBad[category]) > 0:
        wc = WordCloud(width = 800, height = 400)
        wc.generate_from_frequencies(listOfAllWordsBigramBad[category])
        wc.to_file((category + "_bad_Bigram.png"))

Just setting up the whole list of tokens here. Note the regex pattern, set up to drop pure numbers

In [4]:
topic = None

In [30]:
topic = None
newWords = set(['who', 'why', 'how', 'where', 'when', 'what', 'whom'])
messageToken = obtain_Message_Token_List(df, topic, posTag = False, regexPattern = "^[^?a-zA-Z]+", specialNotToRemove = newWords)

# TF-IDF

This section will primarily be for the TF-IDF attempt. I will write another section for LSA.

The process begins as follows:
1. Obtain message token list
2. SOMEWHERE SYNONYMS ARE INVOLVED. WORDNET THIS PART
3. Question words, exclamations? I suspect that these will come in handy. (FIXED exclamations)
4. Token-to-number
5. Matrix it

TF or TF-IDF? I'm inclined to TF here.

Checkpoint:
1 has been done, and so are 3 - 5. I have both TF and TF-IDF out, with options for reducing count to 1.
Last thing to implement are the synonyms/wordnet part.

Last set of sprints:
1. Add in the synonyms/wordnet
    1. After TF-IDFing, or before? A simple fix would be after. Given the list of words, return which words ought to be grouped with each other.
    2. The other option would be to do it before even getting the token list, but in either case pruning is required.
2. Fix code such that ONLY a subset of wordcounts (esp exclamations) get counted as 1 in TF, other words get counted as many as they want

In [7]:
def get_unnested_list(wordList):
    '''
        Gets a un-nested list from a nested list of words
    '''
    if len(wordList) == 0 or (type(wordList[0]) not in set([list, dict, set, tuple])):
        return wordList
    else:
        return get_unnested_list([word for wordArray in wordList for word in wordArray])
    
def get_token_to_index_dict(tokenList, threshold = 1):
    '''
        Gets a dict mapping list of words to tokens given a un-nested list.
    '''
    tokenList = sorted([item[0] for item in Counter(tokenList).items() if item[1] >= threshold])
    return {tokenList[i]:i for i in range(len(tokenList))}

    
def get_ordered_array_of_tokens(tokenToIndexDict):
    ''' Gets back the ordered array of tokens, with the index for each token being the value in the dict.
        Obviously assumes that the dict's values are all unique and range from 0 to len(dict) - 1 '''
    return [indivTuple[0] for indivTuple in sorted(tokenToIndexDict.items(), key=lambda x: x[1])]


def get_TF_matrix(wordList, tokenToIndexDict, onceIsEnough = set([])):
    ## FIXED: NOW matrix is n, p dimensions where n is number of training documents and p rightfully is predictor (word)
    ''' Returns a Numpy Matrix where the (i, j)th position represents the number of times word j is present in document i. 
        Feed it a nested word list please. '''
    nDocuments = len(wordList)
    nWords = len(tokenToIndexDict)
    tfMatrix = np.zeros((nDocuments, nWords), dtype=np.int)
    for i in range(nDocuments):
        messageList = wordList[i]
        for word in messageList:
            if word in tokenToIndexDict:
                j = tokenToIndexDict[word]
                if word in onceIsEnough:
                    tfMatrix[i, j] = 1
                else:
                    tfMatrix[i, j] += 1
    return tfMatrix

    
def get_IDF_score(tfMatrix):
    idfScore = []
    nDocuments = tfMatrix.shape[0]
    for j in range(tfMatrix.shape[1]):
        nAppearingDocs = len(np.where(tfMatrix[:,j] > 0)[0])
        idf = math.log(float(nDocuments) / (nAppearingDocs) + 1)
        idfScore.append(idf)
    return idfScore

def get_TFIDF_matrix(tfMatrix, idfScore, multipleCounts = True, normalized = True):
    ## Assume L2 normalization. I'm kinda lazy to implement L1
    ## MultipleCounts = False means all nonzero iterms in tfMatrix becomes 1
    ## Add functionality to only singularcount a subset of words
    nRows, nCols = tfMatrix.shape
    tfidf = np.zeros(tfMatrix.shape)
    for i in range(nRows):
        for j in range(nCols):
            tfidf[i, j] = float(tfMatrix[i, j] if multipleCounts else min(tfMatrix[i, j], 1)) * idfScore[j]
    if normalized:
        tfidf = normalize(tfidf, axis=1, norm='l2')
    return tfidf

In [31]:
tokenDict10 = get_token_to_index_dict(get_unnested_list(messageToken), threshold = 4)
tokenArray10 = get_ordered_array_of_tokens(tokenDict10)
tfMatrix10 = get_TF_matrix(messageToken, tokenDict10, onceIsEnough = set(["?"]))
idfScore10 = get_IDF_score(tfMatrix10)
tfidf10 = get_TFIDF_matrix(tfMatrix10, idfScore10, normalized = True)
tfidf_v10 = tfidf10
tfMatrix_v10 = tfMatrix10
#tfidf_v10 = tfidf10[~np.all(tfidf10 == 0, axis = 1)]
#tfMatrix_v10 = tfMatrix10[~np.all(tfMatrix10 == 0, axis = 1)]

## LSA/SVD/PCA (kind of the same thing-ish(??!??!))

Now for LSA:

1. SVD
2. Dimensionality Reduction
3. Low-rank reapproximation
4. PCA
5. Percent-variance-explained

Results: Top 10 PVE is 40% for threshold = 10, k = 50. I.e. rather shite

In [21]:
k = 50
U, S, Vt = np.linalg.svd(tfidf_v10, full_matrices=True)
S_prime = np.zeros(tfidf_v10.shape)
S_prime[:k, :k] = np.diag(S[0:k])
reduced_tfidf = np.dot(np.dot(U, S_prime), Vt)


pcaOrig = PCA(n_components = 10)
pcaOrig.fit(reduced_tfidf)
pcaOrig.explained_variance_ratio_[0:10]

array([ 0.05727966,  0.04489612,  0.04324933,  0.03787655,  0.03372424,
        0.03200632,  0.03003233,  0.02899923,  0.02866375,  0.02690317])

## LDA

Going to try LDA now...hold on tight folks

In [14]:
nTopics = 4
random.seed(10)
model = lda.LDA(n_topics=nTopics, n_iter=500, random_state=1)
model.fit(tfMatrix10)
topic_word = model.topic_word_
n = 30
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(tokenArray10)[np.argsort(topic_dist)][:-(n+1):-1]
    print('*Topic {}\n- {}'.format(i, ' '.join(topic_words)))

*Topic 0
- credit card score what interest article balance time pay ? would know like make who point good help how rate thank best limit people used year debt bad when option
*Topic 1
- card costco visa ? new how citi apply amex express american get use what not_have where know reward one want membership nothing need when purchase chase long member offer not_received
*Topic 2
- thanks information thank great helpful info article good informative question much keep read looking work not_answer answer easy not_give business advice wanted many well financial needed detail love service change
*Topic 3
- card credit ? how account need get pay what want number bank like purchase back gas would cash payment one could trying money u address phone know use why new


In [22]:
goodZeroI = 2 ### really based on what we see above
messageOriginal = get_message_set(df, topic)
for i in range(100):
    test = np.random.randint(0, len(messageToken))
    print("Original message " + str(test) + ": " + str(messageOriginal[test]))
    #print("Original category " + str(test) + ": " + str(df["page_vertical_tx"][test]))
    print("Truncated message " + str(test) + ": " + str(messageToken[test]))
    scores = np.dot(model.components_, np.transpose(tfMatrix10[test,:]))
    print("Scores:" + str(scores))
    print("Highest Group: " + str(np.where(np.dot(model.components_, np.transpose(tfMatrix10[test,:])) == max(np.dot(model.components_, np.transpose(tfMatrix10[test,:]))))[0][0]))
    print("Negative Score: " + str(np.nan_to_num(np.true_divide(np.sum(np.delete(scores, goodZeroI)), np.sum(scores)))))
    print("\n")

Original message 1067: Where to apply for Costco anywhere citi card
Truncated message 1067: ['where', 'apply', 'costco', 'anywhere', 'citi', 'card']
Scores:[  7.55195923e-02   2.92078059e-01   5.46721946e-05   5.29095914e-02]
Highest Group: 1
Negative Score: 0.999870002031


Original message 660: Fuck u
Truncated message 660: ['fuck', 'u']
Scores:[  4.71820519e-06   5.45223893e-04   9.11203244e-06   1.03627790e-02]
Highest Group: 3
Negative Score: 0.99916570485


Original message 108: No
Truncated message 108: []
Scores:[ 0.  0.  0.  0.]
Highest Group: 0
Negative Score: 0.0


Original message 537: Great info...will take it into consideration... thanks!!
Truncated message 537: ['great', 'info', 'take', 'consideration', 'thanks']
Scores:[ 0.01087074  0.0021809   0.14765137  0.00438279]
Highest Group: 2
Negative Score: 0.105608268397


Original message 1113: Provide an example of other gas stations where the 4% cash back will apply.
Truncated message 1113: ['provide', 'example', 'gas', u'

## Some fixes that need to be done:

1. ~~Currently the '!' token is too powerful. It nearly forces everything with '!' in it to go to the Positive Comments group, which is nice but doesn't account for exclamations~~ *quick fix fix done Friday 220716*
2. ~~"No", "not" should be added into the permissible lists, and glued to the word after stop-word-filtering. This better captures the "no thanks" situation.~~ *fix done Friday 220716* 
    1. Problem with that is that there are many junk messages that only say "No". I personally think of those as real junk and rather unhelpful for content, so I'm inclined to just throw them away. *fix done Friday 220716. New approach only considers not + NON-STOP real word, so the "no"s are rightfully ignored* 
3. How do we deal with new keywords? In this case what's important for content writers is the specificity of many proper nouns (for example, 'Costco' - points them to the exact card). *current problem: pick out good keywords within top comments*
4. ~~Does a piss-poor job of dealing with anything other than comments of praise. I'm vaguely seeing one that goes "help me apply for credit cards" and a general complaint one, but those are vague.~~ *fix attempted Monday 250716. Simple answer: doesn't matter since all that isn't good is bad, and we can just mine keywords after that.*
5. I need to think of a better way of dealing with comments that have multiple attributions. Currently the two approaches have their drawbacks:
    1. The "classify to max" approach presumes only one topic per comment
    2. The "classify above a certain threshold" approach is heavily contingent on the length of message - more words, higher thresholds.

Work to do with this stuff, but focus on the bottom

1. ~~Understand the LDA output and fix k, the number of topics~~
2. ~~Create an ALERT score~~
3. ~~Do the simple fixes (negation, exclamation etc)~~


Comment-based Sprint (??)

1. Categorize all comments into the n major topics found, ranked by highest scores for that topic
2. Visualize the top few
3. Be able to filter by date range, author, vertical (at the minimum), article (probably for a naive pass post_id)
4. Attach word clouds of each topic
5. Include pie chart of overall sentiment by volume
6. For each comment, provide a page comprising all the relevant scores of topics, and links to relevant stuff (article, author, date, thumbs up/thumbs down etc)
7. Provide links to similar comments (where similar = high cosine similarity)



Article-based Sprint!!

1. Produce the Article-Based Dashboard for one sample article, featuring:
2. An ALERT score/Danger meter for each article, based off key variables (rank the different LDA scores for each article) _** DONE 250716, it's in newSums **_
3. Under each article, feature the top few comments that people are saying (based on LDA scores) _** DONE 250716 **_
4. Also under each article, feature some semblance of a word-cloud (give it a shot)
5. At the lower section, show the pre/post effect of editing articles - simple metrics like:
6. Comment rate/velocity
7. CSAT average pre/post article
8. Average comment ALERT scores


Therefore, things to do include:    
1. Group by article (deal with wrong indexes) _** done **_
2. Calculate important metrics based on the article _** done. summed the non-positive metrics **_
3. **Decide how many categories and what they correspond to - 2 is too few, 3 works better ** 
4. Scale scores and figure out how we should rank articles:
    1. First iteration would be just to sum up all the important category scores (more comments necessarily mean more alerts) (it's good enough)
    2. Second iteration to filter out spam (no actually, it auto-goes away!!)
    3. Third iteration to focus on comment velocity (perhaps calculate only based on latest comments in the *week* etc)
5. Come up with an ALERT score based on all those factors (it can be just simply a 0-1 scale). Auto-flag when something grave happens
    1. We can set rules (eg CSat scores going too low. ASK TOMORROW: what are CSat scores determined by, and where can we fish them from)
6. Highlight important keywords corresponding to "red-flag" topics that are in the article's messages. Those can serve as some form of word-cloud of what to focus on


Regarding the word cloud:
1. Weigh bigrams more than unigrams (we want to know what kind of credit cards are being used)
2. Do we want to compare the percentage of certain words across good and bad scores? Like, if a word appears in both good and bad, and appears more in bad, it will still appear in big words across. 

In [28]:
# Given the same dataframe fed into the tfMatrix, and the LDA model generated, get a data frame listing each post and their relevant scores
def get_topic_scores(df, ldaModel, tfMatrix, groupby = 'post_id'):
    posts = df.groupby('post_id')
    finalTopicScores = np.zeros((len(posts.indices), ldaModel.n_topics + 2)) # each for the topic, the topic id to map back to, and count of topics
    index = 0
    for post_id, tfMatrixIndices in posts.indices.iteritems():
        finalTopicScores[index, 0] = post_id
        finalTopicScores[index, 1:ldaModel.n_topics + 1] = np.sum(np.dot(tfMatrix[tfMatrixIndices,:], np.transpose(ldaModel.components_)), axis = 0)
        finalTopicScores[index, ldaModel.n_topics + 1] = len(tfMatrixIndices)
        index += 1
    return finalTopicScores

def full_topic_scores(tfMatrix, ldaModel):
    return np.dot(tfMatrix, np.transpose(ldaModel.components_))

In [27]:
topicScores = get_topic_scores(df[df["page_vertical_tx"] == topic], model, tfMatrix10)
posts = df[df["page_vertical_tx"] == topic].groupby('post_id')
indivTopicScores = full_topic_scores(tfMatrix10, model)

NameError: name 'get_topic_scores' is not defined

The bottom cell deals with getting the array mapping PAGE_IDs to their "panic scores".

In [16]:
badIndices = range(nTopics)
newdf = df[df["page_vertical_tx"] == topic]
badIndices.remove(goodZeroI)
sums = np.sum(topicScores[:, [indice + 1 for indice in badIndices]], axis = 1)
newSums = np.column_stack((topicScores[:, 0], sums))


In [17]:
top10 = newSums[newSums[:,1].argsort()[::-1]][0:3, :]
#find indiv topic scores for them all:
for i in range(top10.shape[0]):
    index = top10[i, 0]
    tfIndexes = posts.indices[index]
    indivPostScores = indivTopicScores[tfIndexes, :]
    indivPostScoresSum = np.column_stack((tfIndexes, np.sum(indivPostScores[:, badIndices], axis = 1)))
    indivPostScoresSum = indivPostScoresSum[indivPostScoresSum[:,1].argsort()[::-1]][0:40, :]
    print "post_id: " + str(int(index))
    print "url: " + str(newdf['page_path_tx'].iloc[int(indivPostScoresSum[0, 0])])
    print "\n"
    for j in range(indivPostScoresSum.shape[0]):
        print "Message " + str(j + 1) + ": " + replaceString(newdf['message'].iloc[int(indivPostScoresSum[j, 0])])
        print "Score = " + str(indivTopicScores[int(indivPostScoresSum[j, 0]), :])
        print "Normalized Scores = " + str(indivTopicScores[int(indivPostScoresSum[j, 0]), :]/np.sum(indivTopicScores[int(indivPostScoresSum[j, 0]), :]))
        print "\n"

post_id: 227319
url: blog/credit-cards/new-citi-costco-visa-details


Message 1: For my use this card is terrible.  Any card that claims to be an "anywhere" card and charges a rip-off 3% foreign exchange fee is being deceitful and dishonest.  Additionally, in this transfer I am going to experience a credit score drop because the limit on this card is $6,800 less than on my AMEX CostCo card which it replaces.  I will be voicing this loudly to CostCo and to Citi. There may even be a litigious angle to this situation.
Score = [ 0.72427065  0.27320371  0.35678131]
Normalized Scores = [ 0.53481087  0.20173718  0.26345196]


Message 2: It did not address the credit consequences of having to open a VISA credit card account if you don't have one already.  One more credit card account is a hit to the credit score.    No, I won't give up my AMEX.  I HATE VISA!.  As a business owner, they soak you when you process payments.  As a consumer, they spend a fortune on prime time advertising, sports af

In [14]:
for i in range(top10.shape[0]):
    index = top10[i, 0]
    tfIndexes = posts.indices[index]
    indivPostScores = indivTopicScores[tfIndexes, :]
    indivPostScoresSum = np.column_stack((tfIndexes, indivPostScores[:, goodZeroI]))
    indivPostScoresSum = indivPostScoresSum[indivPostScoresSum[:,1].argsort()[::-1]][0:40, :]
    print "post_id: " + str(int(index))
    print "url: " + str(newdf['page_path_tx'].iloc[int(indivPostScoresSum[0, 0])])
    print "\n"
    for j in range(indivPostScoresSum.shape[0]):
        print "Message " + str(j + 1) + ": " + replaceString(newdf['message'].iloc[int(indivPostScoresSum[j, 0])])
        print "Score = " + str(indivTopicScores[int(indivPostScoresSum[j, 0]), :])
        print "\n"

post_id: 227319
url: blog/credit-cards/new-citi-costco-visa-details


Message 1: It did not address the credit consequences of having to open a VISA credit card account if you don't have one already.  One more credit card account is a hit to the credit score.    No, I won't give up my AMEX.  I HATE VISA!.  As a business owner, they soak you when you process payments.  As a consumer, they spend a fortune on prime time advertising, sports affiliations and exclusive deals, which means someone is paying and it is every consumer who pays more for goods and services because the costs get passed down. Love Costco, but may ditch them for another warehouse that accepts AMEX or MasterCard.
Score = [ 0.51997685  0.41329685  0.43476023]


Message 2: For my use this card is terrible.  Any card that claims to be an "anywhere" card and charges a rip-off 3% foreign exchange fee is being deceitful and dishonest.  Additionally, in this transfer I am going to experience a credit score drop because the li

## RAKE

This is an attempt to extract the keywords out from the comments. As results go, currently we have a rather naive version of a comment extractor, that (has already extracted the self-congratulatory comments and) can:
1. Give us back articles weighted on some kind of metric (to validate the quality of this algorithm, we can cross-check with CSat scores)
2. Give us the top few comments that are "concerning" (to validate this is hard, and it will probably involve some form of natural comment weighing).
3. Give us back a set of keywords that are most concerning, and give a sample of a comment with one of those keywords (imagine Yelp? And how it can magically know which food's best)

1, 2 have been done. 3 is the magic one, and for this I'm attempting RAKE:

In [14]:
import RAKE.RAKE as rake

In [15]:
stop_word_list = stopwords.words("english")
stop_word_regex_list = []
for word in stop_word_list:
    word_regex = '\\b' + word + '\\b'
    stop_word_regex_list.append(word_regex)
stop_word_pattern = re.compile('|'.join(stop_word_regex_list), re.IGNORECASE)

In [16]:
def is_acceptable(phrase, min_char_length, max_words_length):

    # a phrase must have a min length in characters
    if len(phrase) < min_char_length:
        return 0

    # a phrase must have a max number of words
    words = phrase.split()
    if len(words) > max_words_length:
        return 0

    digits = 0
    alpha = 0
    for i in range(0, len(phrase)):
        if phrase[i].isdigit():
            digits += 1
        elif phrase[i].isalpha():
            alpha += 1

    # a phrase must have at least one alpha character
    if alpha == 0:
        return 0

    # a phrase must have more alpha than digits characters
    if digits > alpha:
        return 0
    return 1

def generate_candidate_keywords(sentence_list, stopword_pattern, min_char_length=1, max_words_length=5):
    phrase_list = []
    for s in sentence_list:
        tmp = re.sub(stopword_pattern, '|', s.strip())
        phrases = tmp.split("|")
        for phrase in phrases:
            phrase = phrase.strip().lower()
            if phrase != "" and is_acceptable(phrase, min_char_length, max_words_length):
                phrase_list.append(phrase)
    return phrase_list

In [17]:
i = 0
tfIndexes = posts.indices[top10[i, 0]]
sentence_list = " ".join([replaceString(message) for message in newdf['message'].iloc[tfIndexes].tolist()])
phraseList = generate_candidate_keywords(rake.split_sentences(sentence_list), stop_word_pattern, 1, 2)
wordscores = rake.calculate_word_scores(phraseList)

In [18]:
import operator

keywordcandidates = rake.generate_candidate_keyword_scores(phraseList, wordscores)
#for candidate in keywordcandidates.keys():
#    print("Candidate: ", candidate, ", score: ", keywordcandidates.get(candidate))

# sort candidates by score to determine top-scoring keywords
sortedKeywords = sorted(keywordcandidates, key = keywordcandidates.get, reverse=True)
totalKeywords = len(sortedKeywords)

# for example, you could just take the top third as the final keywords
for keyword in sortedKeywords[0:int(totalKeywords / 3)]:
    print("Keyword: \'" + keyword + "\', score: " + str(keywordcandidates[keyword]))

Keyword: 'favorable combination', score: 4.0
Keyword: 'rfid aspect', score: 4.0
Keyword: 'chip reader', score: 4.0
Keyword: 'travel outside', score: 4.0
Keyword: 'credit score', score: 4.0
Keyword: 'executive level', score: 4.0
Keyword: 'another warehouse', score: 4.0
Keyword: 'litigious angle', score: 4.0
Keyword: 'mind redeeming', score: 4.0
Keyword: 'dcc scam', score: 4.0
Keyword: 'reports ytd', score: 4.0
Keyword: 'outrageously high', score: 4.0
Keyword: 'convenience store', score: 4.0
Keyword: 'convenience stores', score: 4.0
Keyword: 'city bank', score: 4.0
Keyword: 'credit consequences', score: 4.0
Keyword: 'exclusive deals', score: 4.0
Keyword: 'annual fees', score: 4.0
Keyword: 'process payments', score: 4.0
Keyword: 'integration problems', score: 4.0
Keyword: 'phone number', score: 4.0
Keyword: 'three month', score: 4.0
Keyword: 'spread like', score: 4.0
Keyword: 'anymore forms', score: 4.0
Keyword: 'waited forever', score: 4.0
Keyword: 'automated payments', score: 4.0
Keywor

TO WORK ON TUESDAY 260716:

1. Figure out a smarter algorithm to capture key questions and key phrases. I'd like to go a bit more complex and perhaps group the questions together and capture bigrams from there...
2. Try wordclouding for the questions
3. I personally think the wordcloud needs more work - instead of what it is right now, there needs to be a marked difference in what kind of words both types of people are asking - so we need some percentage smoothing.


Side notes:
1. Replace all numbers with NUM? 

## Clustering

An approach right now could be trying to group all the questions together by some form of a similarity score (cosine similarity). Of course, two issues:
1. How to determine the number of clusters, k

In [36]:
# Let's try a sample!
i = 0
index = top10[i, 0]
tfIndexes = posts.indices[index]
indivPostScores = np.true_divide(indivTopicScores[tfIndexes, :], np.sum(indivTopicScores[tfIndexes, :], axis = 1)[:,None])
indivPostScoresSum = np.column_stack((tfIndexes, indivPostScores))
indivPostScoresSum = indivPostScoresSum[~np.isnan(indivPostScoresSum[:, 1]), :]
indivPostScores = indivPostScoresSum[:, 1:]

K-means!!! 

In [19]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


array([[ 0.61574313,  0.09067408,  0.29358279],
       [ 0.02418148,  0.92435201,  0.0514665 ],
       [ 0.05137152,  0.03277492,  0.91585356],
       [ 0.95686891,  0.02011644,  0.02301465],
       [ 0.70336269,  0.07819067,  0.21844664],
       [ 0.35304356,  0.34529346,  0.30166298],
       [ 0.51195733,  0.14964107,  0.33840159],
       [ 0.45809076,  0.01568256,  0.52622668],
       [ 0.56122054,  0.22663288,  0.21214658],
       [ 0.20087194,  0.30523251,  0.49389556],
       [ 0.01996544,  0.77779415,  0.20224041]])

In [37]:
i = 0
index = top10[i, 0]
tfIndexes = posts.indices[index]
indivPostScores = np.true_divide(indivTopicScores[tfIndexes, :], np.sum(indivTopicScores[tfIndexes, :], axis = 1)[:,None])
indivPostScoresSum = np.column_stack((tfIndexes, indivPostScores))
indivPostScoresSum = indivPostScoresSum[~np.isnan(indivPostScoresSum[:, 1]), :]
indivPostScores = indivPostScoresSum[:, 1:]

k = int(math.floor(math.sqrt(indivPostScores.shape[0])))
random.seed(10)
km = KMeans(n_clusters=k, n_init=500)
km.fit(indivPostScores)

COMMON_QUESTION_LIMIT = 3
FREQUENT_QUESTION_LIMIT = k # in the hopes that they multiply to number of words
common = Counter(km.labels_).most_common(COMMON_QUESTION_LIMIT)
common = [commonTuple[0] for commonTuple in common if commonTuple[1] > FREQUENT_QUESTION_LIMIT]

indivPostScoresSum2 = np.column_stack((km.labels_, indivPostScoresSum))
for group in common:
    groupdf = indivPostScoresSum2[np.where(indivPostScoresSum2[:, 0] == group), 1:][0]
    minscore = 3
    index = -1
    centroid = km.cluster_centers_[group]
    for i in range(groupdf.shape[0]):
        l1_dist = groupdf[i, 1:] - centroid
        l2_dist = np.dot(l1_dist, np.transpose(l1_dist))
        if l2_dist < minscore:
            index = i
            minscore = l2_dist
    print "Group: " + str(group)
    print "Closest point to centroid: " + str(groupdf[index, 1:])
    messageNew = replaceString(str(newdf['message'].iloc[int(groupdf[index, 0])]))
    print "Message: " + messageNew
    tokenized = [word for word in nltk.word_tokenize(messageNew.lower()) if word not in set(stopwords.words("english"))]
    ## I want proper nouns, verbs.
    print "Tokens: " + str([wordpair[0] for wordpair in nltk.pos_tag(tokenized) if wordpair[1] != "."])

    
        

Group: 7
Closest point to centroid: [ 0.62341501  0.09631664  0.28026834]
Message: where to confirm or set up my new card?
Tokens: ['confirm', 'set', 'new', 'card']
Group: 0
Closest point to centroid: [ 0.49896995  0.15821583  0.34281421]
Message: Just give me a page to put my card numbers so I can activate it. I don't need all the ads
Tokens: ['give', 'page', 'put', 'card', 'numbers', 'activate', "n't", 'need', 'ads']
Group: 4
Closest point to centroid: [ 0.70864551  0.07397952  0.21737497]
Message: When can I apply for the new costco visa card?
Tokens: ['apply', 'new', 'costco', 'visa', 'card']


In [29]:
Counter(km.labels_)

Counter({0: 25,
         1: 13,
         2: 15,
         3: 4,
         4: 11,
         5: 3,
         6: 4,
         7: 40,
         8: 5,
         9: 4,
         10: 7})

In [26]:
COMMON_QUESTION_LIMIT = 3
FREQUENT_QUESTION_LIMIT = k # in the hopes that they multiply to number of words
common = Counter(km.labels_).most_common(COMMON_QUESTION_LIMIT)
common = [commonTuple[0] for commonTuple in common if commonTuple[1] > FREQUENT_QUESTION_LIMIT]
for i in range(indivPostScores.shape[0]):
    if km.labels_[i] == 0:
        print "Message: " + replaceString(str(newdf['message'].iloc[int(indivPostScoresSum[i, 0])]))
        print "Group: " + str(km.labels_[i])
        print "Scores: " + str(indivPostScores[i])
        print "\n"


Message: If Capital One sends me a Visa Card I will use it. I am NOT going to fill out anymore forms for credit cards.
Group: 0
Scores: [ 0.48889247  0.20677161  0.30433592]


Message: It does not give me the a way to set up to confirm receipt of my cards.
Group: 0
Scores: [ 0.51477975  0.15973251  0.32548774]


Message: Confirm receipt card
Group: 0
Scores: [ 0.53634741  0.15147984  0.31217275]


Message: Read the Citibank Costco "BUSINESS" card terms of use.  Re: May be used only for business purchases and may never be used for personal purchases.  You didn't do your homework.  If city bank can show(easy to do) you used the card for any personal purchase they can change the interest rate or deny any and all rebates. Get a copy and read it.  Pretty
Group: 0
Scores: [ 0.5018727   0.14543902  0.35268828]


Message: The APR interest rate is much higher than my AMX card. I will decline the Citi.
Group: 0
Scores: [ 0.49836815  0.14548429  0.35614756]


Message: This card is worthed.
Group:

In [29]:
km.labels_[i]
initial = [cluster.vq.kmeans(indivPostScores,i) for i in range(1,10)]
pyplot.plot([var for (cent,var) in initial])
pyplot.show()

In [122]:
costcodf = newdf.iloc[indivPostScoresSum[:, 0]]
messageToken_n = obtain_Message_Token_List(costcodf, None, posTag = False, regexPattern = "^[^?a-zA-Z]+", specialNotToRemove = newWords)
tokenDict10_n = get_token_to_index_dict(get_unnested_list(messageToken_n), threshold = 2)
tokenArray10_n = get_ordered_array_of_tokens(tokenDict10_n)
tfMatrix10_n = get_TF_matrix(messageToken_n, tokenDict10_n, onceIsEnough = set(["?"]))
idfScore10_n = get_IDF_score(tfMatrix10_n)
tfidf10_n = get_TFIDF_matrix(tfMatrix10_n, idfScore10_n, normalized = True)
tfidf_v10_n = tfidf10_n
tfMatrix_v10_n = tfMatrix10_n

In [133]:
nTopics = 11
random.seed(10)
model2 = lda.LDA(n_topics=nTopics, n_iter=500, random_state=1)
model2.fit(tfMatrix10_n)
topic_word2 = model2.topic_word_
n = 20
for i, topic_dist in enumerate(topic_word2):
    topic_words = np.array(tokenArray10_n)[np.argsort(topic_dist)][:-(n+1):-1]
    print('*Topic {}\n- {}'.format(i, ' '.join(topic_words)))

*Topic 0
- purchase get one question back not_answer who made paying read gasoline store money put rebate every limit card application gas
*Topic 1
- card credit use may one used business service never consumer personal score know limit question how gasoline foreign gas yesterday
*Topic 2
- fee costco foreign transaction travel dcc like thank anywhere customer offered not_worth end line many amex pay accepted interest get
*Topic 3
- card new visa citi how bank mail gmail.com like got want talk compare not_know set ? never gas foreign find
*Topic 4
- costco card visa apply ? citi amex how where not_have would get looking time available nothing when june not_receive business
*Topic 5
- card confirm account how receipt register set not_give pay payment active where give when line know gas etc limit every
*Topic 6
- interest rate great citibank make charge citi purchase rfid not_tell hard change source not_information pretty costco chip gasoline got gas
*Topic 7
- need card want activate k

In [134]:
messageOriginal_n = get_message_set(costcodf, None)
for i in range(30):
    test = np.random.randint(0, len(messageToken_n))
    print("Original message " + str(test) + ": " + str(messageOriginal_n[test]))
    #print("Original category " + str(test) + ": " + str(df["page_vertical_tx"][test]))
    print("Truncated message " + str(test) + ": " + str(messageToken_n[test]))
    print("Scores:" + str(np.dot(model2.components_, np.transpose(tfMatrix10_n[test,:]))))
    print("Highest Group: " + str(np.where(np.dot(model2.components_, np.transpose(tfMatrix10_n[test,:])) == max(np.dot(model2.components_, np.transpose(tfMatrix10_n[test,:]))))[0][0]))
    print("\n")

Original message 75: Thanks for info.
Truncated message 75: ['thanks', 'info']
Scores:[  2.96515938e-04   3.30851944e-04   3.48128808e-04   1.82731841e-04
   1.32934530e-04   3.60685302e-04   4.04448938e-04   3.20256205e-04
   3.42172797e-04   2.57483930e-01   3.54295837e-04]
Highest Group: 9


Original message 86: Just give me a page to put my card numbers so I can activate it. I don't need all the ads
Truncated message 86: ['give', 'page', 'put', 'card', u'number', 'activate', 'not_need', u'ad']
Scores:[ 0.04521868  0.26550868  0.00087032  0.23800822  0.12662014  0.25338142
  0.00101112  0.28903122  0.00085543  0.00091827  0.00088574]
Highest Group: 7


Original message 25: it did not refer to the RFID chip.  Why is it there.
Truncated message 25: ['not_refer', 'rfid', 'chip', 'why']
Scores:[ 0.00044477  0.00049628  0.00052219  0.0002741   0.0001994   0.00054103
  0.06127401  0.00048038  0.05183918  0.00055096  0.01824624]
Highest Group: 6


Original message 49: Which gas stations, r

In [28]:
dir(km)

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_fit_data',
 '_check_test_data',
 '_estimator_type',
 '_get_param_names',
 '_transform',
 'cluster_centers_',
 'copy_x',
 'fit',
 'fit_predict',
 'fit_transform',
 'get_params',
 'inertia_',
 'init',
 'labels_',
 'max_iter',
 'n_clusters',
 'n_init',
 'n_iter_',
 'n_jobs',
 'precompute_distances',
 'predict',
 'random_state',
 'score',
 'set_params',
 'tol',
 'transform',
 'verbose']

In [69]:
lemma = nltk.stem.WordNetLemmatizer()
lemma.lemmatize("scores")

u'score'

In [85]:
pattern = re.compile("^[^a-zA-Z]+")
print (re.match(pattern, "."))

In [87]:
i = 0
df[["message"]].iloc[i, 0] 

'Good information.'

In [23]:
topic = "Credit Cards"
newWords = set(['who', 'why', 'how', 'where', 'when', 'what', 'whom'])
messageToken = obtain_Message_Token_List(df, topic, posTag = False, regexPattern = "^[^?a-zA-Z]+", specialNotToRemove = newWords)

In [24]:
tokenDict10 = get_token_to_index_dict(get_unnested_list(messageToken), threshold = 4)
tokenArray10 = get_ordered_array_of_tokens(tokenDict10)
tfMatrix10 = get_TF_matrix(messageToken, tokenDict10, onceIsEnough = set(["?"]))
idfScore10 = get_IDF_score(tfMatrix10)
tfidf10 = get_TFIDF_matrix(tfMatrix10, idfScore10, normalized = True)
tfidf_v10 = tfidf10
tfMatrix_v10 = tfMatrix10

In [25]:
nTopics = 3
random.seed(10)
model = lda.LDA(n_topics=nTopics, n_iter=500, random_state=1)
model.fit(tfMatrix10)
topic_word = model.topic_word_
n = 30
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(tokenArray10)[np.argsort(topic_dist)][:-(n+1):-1]
    print('*Topic {}\n- {}'.format(i, ' '.join(topic_words)))

*Topic 0
- card costco ? visa new how citi apply get want know amex what express american would one bank need credit reward purchase account when fee use pay back still gas
*Topic 1
- credit thanks thank article score information good card great helpful help info what keep people time understand informative know read bad best debt limit easy advice needed work like much
*Topic 2
- card credit how ? pay what need interest balance question use number year account get one money would much month rate information company used who day not_answer phone like could


In [29]:
goodZeroI = 1 ### really based on what we see above
messageOriginal = get_message_set(df, topic)

topicScores = get_topic_scores(df[df["page_vertical_tx"] == topic], model, tfMatrix10)
posts = df[df["page_vertical_tx"] == topic].groupby('post_id')
indivTopicScores = full_topic_scores(tfMatrix10, model)

badIndices = range(nTopics)
newdf = df[df["page_vertical_tx"] == topic]
badIndices.remove(goodZeroI)
sums = np.sum(topicScores[:, [indice + 1 for indice in badIndices]], axis = 1)
newSums = np.column_stack((topicScores[:, 0], sums)) #newsums contains the post_id, sum of badscores for all associated comments

top10 = newSums[newSums[:,1].argsort()[::-1]][0:3, :]
#find indiv topic scores for them all:
for i in range(top10.shape[0]):
    if top10[i, 1] > 2:
        index = top10[i, 0]
        tfIndexes = posts.indices[index]
        indivPostScores = indivTopicScores[tfIndexes, :]
        indivPostScoresSum = np.column_stack((tfIndexes, np.sum(indivPostScores[:, badIndices], axis = 1)))
        indivPostScoresSum = indivPostScoresSum[indivPostScoresSum[:,1].argsort()[::-1]][0:40, :]
        print "post_id: " + str(int(index))
        print "url: " + str(newdf['page_path_tx'].iloc[int(indivPostScoresSum[0, 0])])
        print "Score: " + str(top10[i, 1])
        print "\n"
        for j in range(indivPostScoresSum.shape[0]):
            if j < 3 and np.sum(indivTopicScores[int(indivPostScoresSum[j, 0]), :]) > 0.4:
                print "Message " + str(j + 1) + ": " + replaceString(newdf['message'].iloc[int(indivPostScoresSum[j, 0])])
                #print "Score = " + str(indivTopicScores[int(indivPostScoresSum[j, 0]), :])
                #print "Normalized Scores = " + str(indivTopicScores[int(indivPostScoresSum[j, 0]), :]/np.sum(indivTopicScores[int(indivPostScoresSum[j, 0]), :]))
                print "\n"
    

    
        

post_id: 227319
url: blog/credit-cards/new-citi-costco-visa-details
Score: 26.6428533047


Message 1: For my use this card is terrible.  Any card that claims to be an "anywhere" card and charges a rip-off 3% foreign exchange fee is being deceitful and dishonest.  Additionally, in this transfer I am going to experience a credit score drop because the limit on this card is $6,800 less than on my AMEX CostCo card which it replaces.  I will be voicing this loudly to CostCo and to Citi. There may even be a litigious angle to this situation.


Message 2: It did not address the credit consequences of having to open a VISA credit card account if you don't have one already.  One more credit card account is a hit to the credit score.    No, I won't give up my AMEX.  I HATE VISA!.  As a business owner, they soak you when you process payments.  As a consumer, they spend a fortune on prime time advertising, sports affiliations and exclusive deals, which means someone is paying and it is every consu

In [ ]:
index = top10[i, 0]
    tfIndexes = posts.indices[index]
    indivPostScores = np.true_divide(indivTopicScores[tfIndexes, :], np.sum(indivTopicScores[tfIndexes, :], axis = 1)[:,None])
    indivPostScoresSum = np.column_stack((tfIndexes, indivPostScores))
    indivPostScoresSum = indivPostScoresSum[~np.isnan(indivPostScoresSum[:, 1]), :]
    indivPostScores = indivPostScoresSum[:, 1:]

    k = int(math.floor(math.sqrt(indivPostScores.shape[0])))
    random.seed(10)
    km = KMeans(n_clusters=k, n_init=500)
    km.fit(indivPostScores)

    COMMON_QUESTION_LIMIT = 3
    FREQUENT_QUESTION_LIMIT = k # in the hopes that they multiply to number of words
    common = Counter(km.labels_).most_common(COMMON_QUESTION_LIMIT)
    common = [commonTuple[0] for commonTuple in common if commonTuple[1] > FREQUENT_QUESTION_LIMIT]

    indivPostScoresSum2 = np.column_stack((km.labels_, indivPostScoresSum))
    for group in common:
        groupdf = indivPostScoresSum2[np.where(indivPostScoresSum2[:, 0] == group), 1:][0]
        minscore = 3
        index = -1
        centroid = km.cluster_centers_[group]
        for i in range(groupdf.shape[0]):
            l1_dist = groupdf[i, 1:] - centroid
            l2_dist = np.dot(l1_dist, np.transpose(l1_dist))
            if l2_dist < minscore:
                index = i
                minscore = l2_dist
        #print "Group: " + str(group)
        #print "Closest point to centroid: " + str(groupdf[index, 1:])
        messageNew = replaceString(str(newdf['message'].iloc[int(groupdf[index, 0])]))
        print "Message: " + messageNew
        tokenized = [word for word in nltk.word_tokenize(messageNew.lower()) if word not in set(stopwords.words("english"))]
        ## I want proper nouns, verbs.
        print "Tokens: " + str([wordpair[0] for wordpair in nltk.pos_tag(tokenized) if wordpair[1] != "."])
        print "\n"
    print "\n"